<a href="https://colab.research.google.com/github/R1A2H1L1/KaggleProject/blob/main/03%20-%20modelo%20con%20preprocesado%20B%20y%20XGBClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
!pip install Unidecode
!pip install xgboost
import unidecode
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.metrics import median_absolute_error, r2_score, mean_squared_error
from progressbar import progressbar as pbar

In [4]:
students_df = pd.read_csv("train.csv")

In [5]:
id_estudiantes = students_df["ID"].copy()

In [6]:
students_df = students_df.drop(['ID'], axis=1)
students_df = students_df.fillna('Miss')

In [7]:
def to_onehot(x):
    values = np.unique(x)
    r = np.r_[[np.argwhere(i==values)[0][0] for i in x]]
    return np.eye(len(values))[r].astype(int)

def replace_column_with_onehot(d, col):
    assert sum(d[col].isna())==0, "column must have no NaN values"
    values = np.unique(d[col])
    k = to_onehot(d[col].values)
    r = d.join(pd.DataFrame(k, columns=["%s_%s"%(col, values[i]) for i in range(k.shape[1])], index=d.index), how='outer')
    del(r[col])
    return r

columns_to_onehot =[
    'F_TIENEINTERNET',
    'E_PAGOMATRICULAPROPIO',
]
onehoted_df = students_df.copy()
for col in columns_to_onehot:
  onehoted_df = replace_column_with_onehot(onehoted_df, col)
onehoted_df

,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,...,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,F_TIENEINTERNET_Miss,F_TIENEINTERNET_No,F_TIENEINTERNET_Si,E_PAGOMATRICULAPROPIO_Miss,E_PAGOMATRICULAPROPIO_No,E_PAGOMATRICULAPROPIO_Si
0,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Técnica o tecnológica incompleta,Si,Si,N,...,0.322,0.208,0.310,0.267,0,0,1,0,1,0
1,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,Técnica o tecnológica completa,Si,No,N,...,0.311,0.215,0.292,0.264,0,1,0,0,1,0
2,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Secundaria (Bachillerato) completa,Si,No,N,...,0.297,0.214,0.305,0.264,0,0,1,0,1,0
3,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,No sabe,Si,No,N,...,0.485,0.172,0.252,0.190,0,0,1,0,1,0
4,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Primaria completa,Si,Si,N,...,0.316,0.232,0.285,0.294,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Secundaria (Bachillerato) completa,Si,No,N,...,0.237,0.271,0.271,0.311,0,0,1,0,0,1
692496,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Primaria incompleta,Si,No,N,...,0.314,0.240,0.278,0.260,0,0,1,0,1,0
692497,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Secundaria (Bachillerato) completa,Si,No,N,...,0.286,0.240,0.314,0.287,0,0,1,0,0,1
692498,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,Primaria completa,No,No,N,...,0.132,0.426,0.261,0.328,0,1,0,0,0,1


In [8]:
def replace_category_for_numbers(d, col, vals_to_change):
  r = d.copy()
  r[col] = r[col].replace(vals_to_change.keys(), vals_to_change.values())
  return r

In [9]:
data_education_dict = {
    'Miss': -2,
    'No Aplica': -1,
    'Ninguno': 0,
    'No sabe': 1,
    'Primaria incompleta': 2,
    'Primaria completa': 3,
    'Secundaria (Bachillerato) incompleta': 4,
    'Secundaria (Bachillerato) completa': 5,
    'Técnica o tecnológica incompleta': 6,
    'Técnica o tecnológica completa': 7,
    'Educación profesional incompleta': 8,
    'Educación profesional completa': 9,
    'Postgrado': 10,
}

data_to_convert = {
    'E_HORASSEMANATRABAJA': {
        'Miss': 0,
        '0': 1,
        'Menos de 10 horas': 2,
        'Entre 11 y 20 horas': 3,
        'Entre 21 y 30 horas': 4,
        'Más de 30 horas': 5,
    },
    'F_ESTRATOVIVIENDA': {
        'Miss': -1,
        'Sin Estrato': 0,
        'Estrato 1': 1,
        'Estrato 2': 2,
        'Estrato 3': 3,
        'Estrato 4': 4,
        'Estrato 5': 5,
        'Estrato 6': 6,
    },
    'RENDIMIENTO_GLOBAL': {
        'bajo': 0,
        'medio-bajo': 1,
        'medio-alto': 2,
        'alto': 3,
    },
    'F_EDUCACIONPADRE':data_education_dict,
    'F_EDUCACIONMADRE':data_education_dict,
}

for col, transform_dict in data_to_convert.items():
    onehoted_df = replace_category_for_numbers(
        onehoted_df,
        col,
        transform_dict)

onehoted_df.head()

/tmp/ipython-input-3900146150.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  r[col] = r[col].replace(vals_to_change.keys(), vals_to_change.values())


,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,...,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,F_TIENEINTERNET_Miss,F_TIENEINTERNET_No,F_TIENEINTERNET_Si,E_PAGOMATRICULAPROPIO_Miss,E_PAGOMATRICULAPROPIO_No,E_PAGOMATRICULAPROPIO_Si
0,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,2,3,6,Si,Si,N,...,0.322,0.208,0.310,0.267,0,0,1,0,1,0
1,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,1,3,7,Si,No,N,...,0.311,0.215,0.292,0.264,0,1,0,0,1,0
2,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,5,3,5,Si,No,N,...,0.297,0.214,0.305,0.264,0,0,1,0,1,0
3,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,1,4,1,Si,No,N,...,0.485,0.172,0.252,0.190,0,0,1,0,1,0
4,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,4,3,3,Si,Si,N,...,0.316,0.232,0.285,0.294,0,0,1,0,1,0


In [10]:
def replace_word(word, sentence: str):
  return sentence.replace(" DE ", " ")

def replace_wrong_words(sentence):
  result = sentence
  wrong_words_dict = {
      '3DEG ': '',
      ' PRIMARIA ': ' ',
      ' PRIMARIA:': ' ',
      'INGENIER?A': 'INGENIERIA',
      'II': 'I',
      'EDUCACI?N': 'EDUCACION',
      'F?SICA': 'FISICA',
      'RECREACI?N': 'RECREACION',
      '?NFASIS': 'ENFASIS',
      'INGL?S': 'INGLES',
      'DISE?O': 'DISENO',
      'COMUNICACI?N': 'COMUNICACION',
      'QU?MICA': 'QUIMICA',
      'MATEM?TICAS': 'MATEMATICAS',
      'EL?CTRICA': 'ELECTRICA',
      'M?SICA': 'MUSICA',
      'DISE??O': 'DISENO',
      'COMUNICACI??N': 'COMUNICACION',
      'QU?MICA': 'QUIMICA',
      'GESTI?N': 'GESTION',
      'GR??FICA': 'GRAFICA',
      'B?SICA': 'BASICA',
      'ADMINISTRACI?N': 'ADMINISTRACION',
      'P?BLICA': 'PUBLICA',
      'FARMAC?UTICA': 'FARMACEUTICA',
      'FILOSOF?A': 'FILOSOFIA',
      'ESC?NICAS': 'ESCENICAS',
      'ECONOM?A': 'ECONOMIA',
      'GASTRONOM?A': 'GASTRONOMIA',
      'GEOLOG?A': 'GEOLOGIA',
      'LOG?STICA': 'LOGISTICA',
      'ART?STICA': 'ARTISTICA',
      'PEDAGOG?A': 'PEDAGOGIA',
      ' DE ': ' ',
      ' EN ': ' ',
      '  ': ' ',
      '.': '',
      '- ': ' ',
  }
  for wrong, well in wrong_words_dict.items():
    if wrong in result: result = result.replace(wrong, well)
  return result


onehoted_df["E_PRGM_ACADEMICO"] = onehoted_df["E_PRGM_ACADEMICO"].map(unidecode.unidecode)
onehoted_df["E_PRGM_ACADEMICO"] = onehoted_df["E_PRGM_ACADEMICO"].map(lambda sentence: sentence.upper())
onehoted_df["E_PRGM_ACADEMICO"] = onehoted_df["E_PRGM_ACADEMICO"].map(replace_wrong_words)
onehoted_df["E_PRGM_DEPARTAMENTO"] = onehoted_df["E_PRGM_DEPARTAMENTO"].map(unidecode.unidecode)
onehoted_df

,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,...,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,F_TIENEINTERNET_Miss,F_TIENEINTERNET_No,F_TIENEINTERNET_Si,E_PAGOMATRICULAPROPIO_Miss,E_PAGOMATRICULAPROPIO_No,E_PAGOMATRICULAPROPIO_Si
0,20212,ENFERMERIA,BOGOTA,Entre 5.5 millones y menos de 7 millones,2,3,6,Si,Si,N,...,0.322,0.208,0.310,0.267,0,0,1,0,1,0
1,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,1,3,7,Si,No,N,...,0.311,0.215,0.292,0.264,0,1,0,0,1,0
2,20203,MERCADEO Y PUBLICIDAD,BOGOTA,Entre 2.5 millones y menos de 4 millones,5,3,5,Si,No,N,...,0.297,0.214,0.305,0.264,0,0,1,0,1,0
3,20195,ADMINISTRACION EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,1,4,1,Si,No,N,...,0.485,0.172,0.252,0.190,0,0,1,0,1,0
4,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,4,3,3,Si,Si,N,...,0.316,0.232,0.285,0.294,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,3,2,5,Si,No,N,...,0.237,0.271,0.271,0.311,0,0,1,0,0,1
692496,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,5,3,2,Si,No,N,...,0.314,0.240,0.278,0.260,0,0,1,0,1,0
692497,20183,ADMINISTRACION SALUD OCUPACIONAL,BOGOTA,Entre 1 millón y menos de 2.5 millones,2,3,5,Si,No,N,...,0.286,0.240,0.314,0.287,0,0,1,0,0,1
692498,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,2,1,3,No,No,N,...,0.132,0.426,0.261,0.328,0,1,0,0,0,1


In [12]:
onehoted_df = onehoted_df.drop(['F_TIENEINTERNET_Miss'], axis=1)
print (onehoted_df.shape)
onehoted_df

(692500, 23)


,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,...,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,F_TIENEINTERNET_No,F_TIENEINTERNET_Si,E_PAGOMATRICULAPROPIO_Miss,E_PAGOMATRICULAPROPIO_No,E_PAGOMATRICULAPROPIO_Si
0,20212,ENFERMERIA,BOGOTA,Entre 5.5 millones y menos de 7 millones,2,3,6,Si,Si,N,...,2,0.322,0.208,0.310,0.267,0,1,0,1,0
1,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,1,3,7,Si,No,N,...,0,0.311,0.215,0.292,0.264,1,0,0,1,0
2,20203,MERCADEO Y PUBLICIDAD,BOGOTA,Entre 2.5 millones y menos de 4 millones,5,3,5,Si,No,N,...,0,0.297,0.214,0.305,0.264,0,1,0,1,0
3,20195,ADMINISTRACION EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,1,4,1,Si,No,N,...,3,0.485,0.172,0.252,0.190,0,1,0,1,0
4,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,4,3,3,Si,Si,N,...,1,0.316,0.232,0.285,0.294,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,3,2,5,Si,No,N,...,2,0.237,0.271,0.271,0.311,0,1,0,0,1
692496,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,5,3,2,Si,No,N,...,0,0.314,0.240,0.278,0.260,0,1,0,1,0
692497,20183,ADMINISTRACION SALUD OCUPACIONAL,BOGOTA,Entre 1 millón y menos de 2.5 millones,2,3,5,Si,No,N,...,1,0.286,0.240,0.314,0.287,0,1,0,0,1
692498,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,2,1,3,No,No,N,...,0,0.132,0.426,0.261,0.328,1,0,0,0,1


In [17]:
final_student_df = replace_column_with_onehot(onehoted_df, 'E_PRGM_DEPARTAMENTO')
y = final_student_df["RENDIMIENTO_GLOBAL"].values
X = final_student_df.drop(['RENDIMIENTO_GLOBAL'], axis=1)
X.E_PRGM_ACADEMICO = X.E_PRGM_ACADEMICO.astype("category")
X.E_VALORMATRICULAUNIVERSIDAD = X.E_VALORMATRICULAUNIVERSIDAD.astype("category")
X.PERIODO_ACADEMICO = X.PERIODO_ACADEMICO.astype("category")
del final_student_df
print (X.shape, y.shape)

(692500, 52) (692500,)


In [18]:
id_estudiantes = id_estudiantes.loc[X.index]

In [19]:
def fit_and_score(estimator, X_train, X_test, y_train, y_test):
    """Fit the estimator on the train set and score it on both sets"""
    estimator.fit(X_train, y_train, eval_set=[(X_test, y_test)])

    train_score = estimator.score(X_train, y_train)
    test_score = estimator.score(X_test, y_test)

    return estimator, train_score, test_score

In [20]:
# Convertir columnas tipo object a category
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].astype('category')

In [21]:
Xtr, Xts, ytr, yts, id_tr, id_ts = train_test_split(X, y, id_estudiantes, test_size=0.3, random_state=42)
estimator = xgb.XGBClassifier(
    random_state=42,
    subsample=0.5,
    learning_rate=0.01,
    max_depth=11,
    n_estimators=4,
    tree_method="approx",
    colsample_bynode=0.8,
    num_parallel_tree=20,
    enable_categorical=True)
estimator, train_score, test_score = fit_and_score(estimator, Xtr, Xts, ytr, yts)
print ("train score %.3f"%(train_score))
print ("test score  %.3f"%(test_score))

[0]	validation_0-mlogloss:1.38407
[1]	validation_0-mlogloss:1.38188
[2]	validation_0-mlogloss:1.37975
[3]	validation_0-mlogloss:1.37763
train score 0.448
test score  0.423


In [22]:
y_pred = estimator.predict(Xts)

In [23]:
mapeo = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
y_pred_etiquetas = pd.Series(y_pred).map(mapeo)


In [24]:

resultado_df = pd.DataFrame({
    "ID": id_ts.values,
    "RENDIMIENTO_GLOBAL_PREDICHO": y_pred_etiquetas
})

resultado_df.to_csv("predicciones_estudiantes.csv", index=False)
